# Physics Informed Neural Network for steady 1D C-D Nozzle Problem

## Problem Statement

* Create a Physics informed Neural Network or **'PINN'** in short, which can replicate the effect of **changing back pressure** of a steady 1D converging-diverging nozzle, and __accurately predict the physical state of fluid at any point through the nozzle length.__*

* To train a Neural network based on the parameters to be input: *Back Pressure* and x (point at which output quantities are desired) and accurately predict *density* (rho), *pressure* (P), *speed* (u) and *specific energy* (E) at that point. But this would just give an ordinary deep learning network trained entirely based on the data, acting as a black box which just takes 2 inputs and produces 4 outputs. What we desire is a 'Physics Informed' neural network, meaning it abides the laws of Phyics apart from just matching ground truth output and predicted output.

* Assume a converging-diverging nozzle with cross section Area given by:

\begin{align}
\textbf{S}(x) &= 1 + 2.2(3 x - 1.5)^2 \\
\end{align}

   where **x** is the distance from left end of the nozzle, *range (0,1)*


<img src = "http://www.dept.aoe.vt.edu/~devenpor/aoe3114/CD%20Nozzle%20Sim/fig1.gif">

## The governing conservative form Euler equation to solve steady 1D Nozzle problem numerically:

### Solve numerically and generate data:

\begin{equation*}
\frac{\partial [S\textbf{u}]}{\partial t} + \frac{\partial [S\textbf{F}]}{\partial x} - \textbf{B}\frac{\partial S}{\partial x} = 0
\end{equation*}

### where
\begin{equation*}
\textbf{u} = 
\begin{bmatrix}
\rho \\
\rho u \\
\rho E
\end{bmatrix},
\;\;\;\;\;\;\;\;\;\;\;\;\;\;
\textbf{F} = 
\begin{bmatrix}
\rho u \\
\rho u^2 + P \\
(\rho E + P)u
\end{bmatrix}
\;\;\;\;\;\;\;\;\;\;\;\;\;\;
\textbf{B} = 
\begin{bmatrix}
0 \\
P \\
0
\end{bmatrix}
\end{equation*}

- The above equation is used in a higher order scheme of 100 data point 1D mesh in order to generate data with the use of appropriate boundary conditions. 

## Solution Brief

* To create a neural network trained using the above data set, which also abides the Laws of Physics as provided by the Euler equations.

* The first part of the solution is simple, create a basic deep learning framework and test your network. Note that we have to divide the dataset into training set and test set first and then perform the training on the training data set. For plotting the results and testing the performance alone would the test data set be used.

* The second part demands Physics to be taught to the network. We want the network to train, while following the governing equations at each iteration step. This can be implemented by minimizing the residuals generated when the steady state expressions are evaluated from the predicted output.

### Steady equation written in terms of fundamental quantitites:

\begin{equation*}
\frac{\partial (\rho u S)}{\partial x} = 0\quad\quad\quad\quad\quad\text{.. (1)} \\
\end{equation*}

\begin{equation*}
\frac{\partial ((\rho u^2 + P)S)}{\partial x} - P\frac{\partial S}{\partial x} = 0\quad\text{.. (2)}\
\end{equation*}

\begin{equation*}
\frac{\partial ((\rho E + P)uS)}{\partial x} = 0\quad\quad\quad\text{.. (3)}
\end{equation*}

**Remember**, since the training set itself is dicretized and plus our network has inaccuracies, there will always be a residual when we actually put the values of output in the above equations, or in other words, it can never be absolute 0.

### Residuals and new loss function:

\begin{equation*}
\frac{\partial (\rho u S)}{\partial x} = e1 \\
\end{equation*}

\begin{equation*}
\frac{\partial ((\rho u^2 + P)S)}{\partial x} - P\frac{\partial S}{\partial x} = e2
\end{equation*}

\begin{equation*}
\frac{\partial ((\rho E + P)uS)}{\partial x} = e3
\end{equation*}

* In addition to reducing the Mean Squared Error (MSE) of the ground truth outputs and predicted outputs of the 4 parameter variables, we try to minimize the squared error residuals generated by the steady state governing equations.


* The __loss__ will now be calculated with additional terms: **e<sup>2</sup>** + ..

    i.e.
\begin{equation*}
\textbf{loss} = MSE + e_{1}^{2} + e_{2}^{2} + e_{3}^{2}
\end{equation*}

* Ideally if the loss goes to zero, it means that all the square terms are zero individually. This means that not only is the predicted output exaclty matching the corresponding ground truth but also for each data point of the dataset, the steady state governing equations are satisfied. This is the ideal Physics informed Neural network. 


* But it cannot happen due to the reason mentioned before, and the network will converge loss to a very small value impying being trained and 'Physics informed'.

### But how do we differentiate the network outputs with respect to x?

To calculate residuals, we need the partial x derivatives of outputs or some combination of outputs.

* We use a technique used by back propagation, which is basically chain rule to find exact derivatives of output variables with respect to all weights.


* This method is called Automatic Differential (AutoDiff) and forms the very backbone of training deep learning networks. Backprop is the most common application of AutoDiff.


\begin{equation*}
\textbf{tf.gradients(a,b)} = \frac{\partial a}{\partial b}
\end{equation*}

    For example:
\begin{equation*}
e_{1} = \frac{\partial (\rho u S)}{\partial x} = tf.gradients(\rho uS, x)
\end{equation*}

Likewise we find the residuals and add the squared errors into the loss function expression for the optimizer to minimize.

### PINN respresentation

<img src="PINN.png">

## Comparison of CFD and Physics Informed Neural Network Results in Shock condition

***The PINN has worked pretty well without any manual optimization required, the network got trained in less than 3 minutes and results are satisfactorily close.***

### Relative test erros of prediction by PINN:

1. **Test Error in Pressure is** 0.020054826925608404

2. **Test Error in density is** 0.01263928640444378

3. **Test Error in speed is** 0.03338287835830078

4. **Test Error in specific Energy is** 0.006931544294609289

### Plots

<img src = "Figure_1.png">

<img src = "Figure_2.png">

<img src = "Figure_3.png">

<img src = "Figure_4.png">

## THANK YOU